<a href="https://colab.research.google.com/github/inspire-lab/SecurePrivateAI/blob/master/8_fl_and_sl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Privacy Preserving Machine Learning

First things first. Let's run the package installations. They take quite a while.

Add the end of the installation you need to hit the restart button. 


In [0]:
!pip install tensorflow_federated pysyft==0.2.5

Next we'll get our usual boilerplate code out of the way. Data loading, splitting, etc.

Load our data set and split it into test and training data.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

n_instances = 32 # the number of instance each party has
n_parties = 10 # number of parties

# load data and transform it
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = np.concatenate( [ x_train[ y_train == 0 ][ :200 ], x_train[ y_train == 1 ][ :200 ] ] ).reshape( (400,-1) )
x_test = np.concatenate( [ x_test[ y_test == 0 ][ :200 ], x_test[ y_test == 1 ][ :200 ] ] ).reshape( (400,-1) )
x_train = x_train.astype( float ) / 255.
x_test = x_test.astype( float ) / 255.

print( 'training data: ', x_train.shape )
print( 'test data: ', x_test.shape )

# labels
y_train = np.concatenate( [ np.zeros( 200 ), np.ones( 200 ) ] )
y_test = np.concatenate( [ np.zeros( 200 ), np.ones( 200 ) ] )

idx = np.arange( len( x_train ) )
np.random.shuffle( idx )
x_train = x_train[ idx ]
y_train = y_train[ idx ]

# Federated Learning

Federated Learning is the idea of sending the model to multiple clients. The clients train on their own data and send the updates back to the server. On the server the individual updates are aggregated.

![The pipeline](https://github.com/inspire-lab/SecurePrivateAI/raw/main/images/FL.png)

We will pretend that data is sent between the parties but for simplicity the data never leaves this process.

We are using a simple perceptron as our learning algorithm. We will assume we have 10 parties and give them all small sub set of the data. The goal of the perceptron is to distinguish 0s and 1s from the MNIST data set.

The perceptron algorithm is simple for an input vector $x$,  the weight vector $w$ and the intercept $b$
the output $y$ is calculated as:

$$ y = \sum_{i=1}^{n} x_i w_i $$

In [0]:
# first practice to do
# gather the data for each party
data_x = [  ]
data_y = [  ]


Now that we have our data let's tackle the learning problem in a none federated setting for a single party.


In [0]:
# initialize the weights of our perceptron
weights =
 
# learning rate
lr = 0.1
# batch size
bs = 16

# the training function
def train( weights, x, y ):
  # train for a number of epochs
  for e in range( 10 ):
    acc = [] 
    print( 'epoch', e )
    # the actual sgd
    for i in range( len( x ) // bs ):
      batch_x = x[ i * bs : (i+1) * bs ]
      batch_y = y[ i * bs : (i+1) * bs ]
      
      # forward step
      f =
      
      # apply sigmoid
      f = 
      
      # calculate the error
      error = 
      # calculate gradient
      g = 
      
      # weight update 
      weights = 
      # calculate acc
      preds = 
      batch_acc = 
      acc.append( batch_acc )
    print( 'acc: ', sum( acc ) / len( acc ) )
    
  # test data
  prediction =
  print( 'test acc:',

train( weights, data_x[ 0 ], data_y[ 1 ] )


Now that we have solved it for one party lets modify the code to send the gradient updates to a central party that aggregates it and updates the weights.

In [0]:
# the local weights of every party 
models = 

def get_gradient( weights, x, y ):
  """
  returns the gradients wrt. loss for the training samples
  """
  grads = []
  for i in range( len( x ) // bs ):
    batch_x = x_train[ i * bs : (i+1) * bs ]
    batch_y = y_train[ i * bs : (i+1) * bs ]
    # forward step
    f = 
    # sigmoid
    f = 
    # loss
    error = 
    # gradient
    g =
    # save the gradients from this batch 
    grads.append( g )  
  
  # return the average of the gradients
  return 

# the servers weights
w =
# do 10 epochs
for i in range( 10 ):
  # get all gradients from the parties
  gradients = [ get_gradient( models[ i ], data_x[ i ], data_y[ i ] ) for i in range( n_parties ) ]

  # average gradients
  gradients =
  # do weight updates
  w -=
  # test data
  prediction =
  print( 'test acc:',


The problem with this solution is that their aggregator can see the weights of the individual parties. This is a potential information leak.

A solution to this problem is applying a mask to the data. But since we still want to have usable gradients we need a way to remove the mask again. We assume that the clients have formed pairs and securely shared a mask between them. Each client has been assigned a unique id as well. The masking process is as follows:


1.  The client with the smaller id adds the mask to the gradient
2.  The client with the larger id subtracts the mask from the gradient
3. When gradients are aggregated by the server the masks will cancel each other out and only the sum of the gradients remains

For simplicity, we are not making the protocol resistant to dropouts. This would require forming pairs between all clients and additional masking to ensure the privacy of the data.




In [0]:
# the local weights of every party 
models =

# shared masks. 0 and 5, 1 and 6, etc share a mask
masks = 



# learning rate
lr = 0.1
# batch size
bs = 16

def get_gradient_masked( weights, x, y, id ):
  grads = []
  for i in range( len( x ) // bs ):
    batch_x = x_train[ i * bs : (i+1) * bs ]
    batch_y = y_train[ i * bs : (i+1) * bs ]
    # forward step
    f = 
    # sigmoid
    f = 
    # loss
    error = 
    # gradient
    g =
    grads.append( g )  

  g = 
  # add the mask

  return g

w 
for i in range( 10 ):
  # get all gradients
  gradients = [ get_gradient_masked( models[ i ], data_x[ i ], data_y[ i ], i ) for i in range( n_parties ) ]

  # average gradients
  gradients = 
  # do weight updates
  w = 
  # test data
  prediction = 
  print( 'test acc:', 

## Federated Learning with TensorFlow

Out of the pieces that we have assembled above we can build a federated learning algorithm. An alternative is to use the implementation that is provided by TensorFlow. Below is code that trains a model on distributed data using secure aggregation.

In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = np.concatenate( [ x_train[ y_train == 0 ], x_train[ y_train == 1 ] ] )
x_test = np.concatenate( [ x_test[ y_test == 0 ], x_test[ y_test == 1 ] ] )
x_train = x_train.astype( float ) / 255.
x_test = x_test.astype( float ) / 255.

x_train = x_train.reshape( ( x_train.shape[ 0 ], -1  ) )
x_test = x_test.reshape( ( x_test.shape[ 0 ], -1  ) )
print( x_train.shape )

# labels
y_train = np.concatenate( [ np.zeros( np.sum( y_train == 0 ) ), np.ones( np.sum( y_train == 1  ) ) ]  )
y_test = np.concatenate( [ np.zeros( np.sum( y_test == 0 ) ), np.ones( np.sum( y_test == 1 ) ) ]  ) 


In [0]:
import collections
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

tf.get_logger().setLevel('INFO')

# parameters
NO_CLIENTS = 3 # number of clients
TOTAL_SAMPLES = x_train.shape[ 0 ]
NO_CLIENT_SAMPLES = TOTAL_SAMPLES // NO_CLIENTS # number of samples per client
BATCH_SIZE = 32
EPOCHS = 10

# shuffle data
idx = np.arange( TOTAL_SAMPLES )
np.random.shuffle( idx )
x_train = x_train[ idx ]
y_train = y_train[ idx ]

data = []
# split into clients
for i in range( NO_CLIENTS ):
  x = x_train[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES  ]
  print( x.shape )
  y = y_train[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES ].reshape( [-1,1] ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
  ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )


# define a function that builds our model
def build_model():
  model = tf.keras.models.Sequential()

  model.add( tf.keras.layers.Dense( 64, activation='relu', input_shape=( x_train.shape[ 1: ] ) ) )
  model.add( tf.keras.layers.Dense( 2, activation='softmax' ) )

  return model

def model_function():
  # we need a dummy batch to build the federated model
  # From the docs:
  # A nested structure of values that are convertible to batched tensors
  # with the same shapes and types as expected by forward_pass(). 
  # The values of the tensors are not important and can be filled with any 
  # reasonable input value.
  dummy_batch = collections.OrderedDict( [ 
      ('x', np.ones( ( BATCH_SIZE, x_train.shape[ 1 ] ) ) ),
      ('y', np.ones( ( BATCH_SIZE, 1) ) ) ] )

  # get the compiled keras model
  model = build_model()
  # use tensorflow function to create a federated learning model
  return tff.learning.from_keras_model( model, loss=tf.keras.losses.SparseCategoricalCrossentropy(),  dummy_batch=dummy_batch, metrics=[tf.keras.metrics.SparseCategoricalAccuracy() ] )


# use tensorflow to create the averaging algorithm
algorithm = tff.learning.build_federated_averaging_process( model_function, client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02 ) ) 

# initialize the learning algorithm and get the initial state
state = algorithm.initialize()

# run the training steps
for e in range( EPOCHS ):
  state, metrics = algorithm.next( state, data )
  print( 'epoch' , e , metrics )



# Split Learning and Pysyft

Split learning considers from another perspective

![pipeline](https://github.com/inspire-lab/SecurePrivateAI/raw/main/images/SL.png)

In [0]:
import torch
import syft as sy

# allow pysyft to work its magic on torch tensors
hook = sy.TorchHook(torch)

# create a virtual worker. in an actual setting this would be on a different machine
client = sy.VirtualWorker( hook, id='client' )

# define a tensor and send it to the client
x = torch.tensor([1,2,3,4,5])
# this leaves us with a pointer to the tensor
x_pointer = x.send( client )

# check out some meta data
print( x_pointer )
print( client._objects )

# we can use this pointers like normal tensors
result = x_pointer + x_pointer
print( result )

# if we want the result we can call get() to send the tensor back to us
result_local = result.get()
# once we call get() it removes the tensor from the other side and our pointer
# becomes invalid
print( result_local )
print( client._objects )
# print( result )

In [0]:
import torch
from torchvision import datasets, transforms
from torch import nn, optim
import syft as sy
hook = sy.TorchHook(torch)

epochs = 5

# Data preprocessing
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])
train_set = datasets.MNIST('mnist', download=True, train=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)

torch.manual_seed(0)

# Define our model segments

input_size = 784
hidden_sizes = [128, 640]
output_size = 10

models = [
    nn.Sequential(
                nn.Linear(input_size, hidden_sizes[0]),
                nn.ReLU(),
                nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                nn.ReLU(),
    ),
    nn.Sequential(
                nn.Linear(hidden_sizes[1], output_size),
                nn.LogSoftmax(dim=1)
    )
]

# Create optimisers for each segment and link to their segment
optimizers = [
    optim.SGD(model.parameters(), lr=0.03,)
    for model in models
]

# create some workers
alice = sy.VirtualWorker(hook, id="alice")
bob = sy.VirtualWorker(hook, id="bob")
workers = alice, bob

# Send Model Segments to starting locations
model_locations = [alice, bob]

for model, location in zip(models, model_locations):
    model.send(location)

def train(x, target, models, optimizers):
    # Training Logic

    #1) erase previous gradients (if they exist)
    for opt in optimizers:
        opt.zero_grad()

    #2) make a prediction
    a = models[0](x)

    #3) break the computation graph link, and send the activation signal to the next model
    remote_a = a.move(models[1].location, requires_grad=True)

    #4) make prediction on next model using received signal
    pred = models[1](remote_a)

    #5) calculate how much we missed
    criterion = nn.NLLLoss()
    loss = criterion(pred, target)

    #6) figure out which weights caused us to miss
    loss.backward()

    # 7) send gradient of the received activation signal to the model behind
    # grad_a = remote_a.grad.copy().move(models[0].location)

    # 8) backpropagate on bottom model given this gradient
    # a.backward(grad_a)

    #9) change the weights
    for opt in optimizers:
        opt.step()

    #10) print our progress
    return loss.detach().get()

for i in range(epochs):
    running_loss = 0
    for images, labels in train_loader:
        images = images.send(alice)
        images = images.view(images.shape[0], -1)
        labels = labels.send(bob)
        
        loss = train(images, labels, models, optimizers)
        running_loss += loss

    else:
        print("Epoch {} - Training loss: {}".format(i, running_loss/len(train_loader)))


More references:

https://learnopencv.com/federated-learning-using-pytorch-and-pysyft/

https://github.com/OpenMined/PySyft/tree/syft_0.2.x/examples/tutorials/advanced/split_neural_network